In [4]:
import pandas as pd

import numpy as np
import numpy.linalg as ln

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

torch.manual_seed(42)

Create classes for handling models and dataset in pytorch

In [5]:
class Net(torch.nn.Module):
    def __init__(self, inputSize, n_hidden, n_hidden2, n_hidden3, outputSize):
        super(Net, self).__init__()
        self.input_layer = torch.nn.Linear(inputSize, n_hidden)
        self.hiddens = nn.ModuleList(
            nn.Linear(n_hidden, n_hidden2),
            nn.Linear(n_hidden2, n_hidden3)
        )
        self.output_layer = torch.nn.Linear(n_hidden3, outputSize),
        
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for hidden in self.hiddens:
            x = F.relu(hidden)
        x = self.output_layer(x)
        return x


class EnronDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        if not torch.is_tensor(y):
            y = torch.from_numpy(y)
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

preprocess and produce train and test csv files from the entire dataframe 

In [ ]:
df = pd.read_csv(
    'data/emails.csv',
    )

shuffle the samples and creating training and test sets

In [ ]:
df_test = df.sample(frac=0.2, random_state=99) # TODO this should be a specific test set taken from mails in a specific range 
df_train = df.drop(df_test.index)

x_train = df_train[train_cols].to_numpy()
y_train = df_train[target_col].to_numpy()

x_test = df_test[train_cols].tu_numpy()
y_test = df_test[target_col].to_numpy()

dataset = EnronDataset(x, y)
trainloader = torch.utils.data.DataLoader(
    dataset, 
    batch_size=2**10, # TODO check this
    shuffle=True, 
    num_workers=1 # TODO check this
    )

initializing model

In [ ]:
inputDim = len(col2use)
outputDim = 1
learningRate = 1e-4
epochs = 3000
n_hidden = 375
n_hidden2 = 187
n_hidden3 = 93
net = Net(inputDim, n_hidden,n_hidden2,n_hidden3, outputDim)
loss_f = torch.nn.MSELoss() #mean squared error
optimizer = torch.optim.Adam(net.parameters(), lr = learningRate) 

start training

In [ ]:
loss=0
current_loss=0
loss_list=[]
for epoch in range(0, epochs):
    
    for i,data in enumerate(trainloader,0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape(((targets.shape[0], 1)))
        
        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss= loss_f(outputs, targets)
        
        loss.backward()
        current_loss += loss.item()
        optimizer.step()
        
    if(epoch%1==0 or epoch==epochs-1):
        l = current_loss/i
        print('epoch {}, loss {}'.format(epoch, l))
        loss_list.append(l)
        current_loss=0

Predict

In [ ]:
with torch.no_grad():
    predicted = net(x_test)

y_test = y_test.detach().numpy()
predicted = predicted.reshape(-1,1)
predicted = predicted.detach().numpy()
predicted = np.int_(predicted)
predicted = predicted.T[0]

i=0
for a in predicted:
    if a<0:
        predicted[i]=0
    i+=1

print("PREDETTI")
print(predicted)
print("VERI")
print(y_test)

Err=(ln.norm(y_test-predicted,ord=2)/ln.norm(y_test,2))*100
print("Errore")
print(Err)

Graphics

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Mail predette")
plt.plot(range(len(predicted)),predicted,'g')
plt.savefig("img/hour/predicted_A10B10.png",dpi=72)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Mail reali (set di test)")
plt.plot(range(len(y_test)),y_test,'c')
plt.savefig("img/hour/ytest_A10B10.png",dpi=72)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Confronto tra valori reali e predetti su 300 campioni")
plt.plot(range(len(y_test[28100:28400])),y_test[28100:28400],'c')
plt.plot(range(len(predicted[28100:28400])),predicted[28100:28400],'g')
plt.legend(["Valori reali","Valori predetti"])
plt.savefig("img/hour/comparison_A10B10.png",dpi=72)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Loss")
plt.xlabel("Steps")
plt.ylabel("Mean squared error")
plt.plot(range(len(loss_list)),loss_list,'b')
plt.grid(which='both')
plt.savefig("img/hour/loss_A10B10.png",dpi=72)
plt.show()